In [1]:
import tensorflow as tf

#### 1. 定义变量及滑动平均类
* [class tf.train.ExponentialMovingAverage](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/train/ExponentialMovingAverage#__init__)
* __init__(decay, num_updates=None, zero_debias=False, name='ExponentialMovingAverage')
* $${shadow\_variable} = decay * shadow\_variable + (1 - decay) * variable$$
* when num_updates aviliable:
$$decay=min\lbrace{decay},\frac{1+num\_updates}{10+num\_updates}\rbrace$$

In [2]:
# 定义一个变量计算滑动平均
v1 = tf.Variable(0, dtype=tf.float32)
# 模拟迭代次数，用于动态控制衰减率（ｄｅｃａｙ）
step = tf.Variable(0, trainable=False)
# class
ema = tf.train.ExponentialMovingAverage(0.99, step)
# 定义一个更新滑动平均的操作。
maintain_averages_op = ema.apply([v1]) 

#### 2. 查看不同迭代中变量取值的变化。

In [3]:
with tf.Session() as sess:
    
    # 初始化
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    # print `v1` & `v1的滑动平均`　all 0
    print sess.run([v1, ema.average(v1)])
    
    # 更新变量v1的取值为５
    sess.run(tf.assign(v1, 5))
    # 更新滑动平均，decay=min(0.99,(1+step)/(10+step)=0.1)=0.1
    # average(v1) = decay*shadow_variable+(1-decay)*variable = 0.1*0+0.9*5=4.5
    sess.run(maintain_averages_op)
    print sess.run([v1, ema.average(v1)]) 
    
    # 更新step和v1的取值
    sess.run(tf.assign(step, 10000))  
    sess.run(tf.assign(v1, 10))
    # 更新滑动平均，decay=min(0.99,(1+step)/(10+step)=0.1)=0.99
    # average(v1) = decay*shadow_variable+(1-decay)*variable = 0.99*4.5+0.01*10=4.555
    sess.run(maintain_averages_op)
    print sess.run([v1, ema.average(v1)])       
    
    # 更新一次v1的滑动平均值
    # 更新滑动平均，decay=min(0.99,(1+step)/10+step=0.1)=0.99
    # average(v1) = decay*shadow_variable+(1-decay)*variable = 0.99*4.555+0.9*10=4.60945
    sess.run(maintain_averages_op)
    print sess.run([v1, ema.average(v1)])       


[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.555]
[10.0, 4.60945]
